In [1]:
import tvm
from tvm import te
import numpy as np
import d2ltvm
import os

target = 'llvm'

successd...



In [6]:
c, n, tc = 4, 2, 2
x = np.arange(c * n * n).reshape((c, n, n)).astype('float32')
print('x\n',x)
y = x.reshape(c // tc, n, n, tc).transpose(0, 2, 3, 1)
print('y\n',y.shape,'\n', y)

x
 [[[ 0.  1.]
  [ 2.  3.]]

 [[ 4.  5.]
  [ 6.  7.]]

 [[ 8.  9.]
  [10. 11.]]

 [[12. 13.]
  [14. 15.]]]
y
 (2, 2, 2, 2) 
 [[[[ 0.  4.]
   [ 1.  5.]]

  [[ 2.  6.]
   [ 3.  7.]]]


 [[[ 8. 12.]
   [ 9. 13.]]

  [[10. 14.]
   [11. 15.]]]]


In [4]:
def conv_pack(oc, ic, nh, nw, kh, kw, ph, pw, toc, tic):
    """ Pack data and weight for convolution
        oc, ic : output and input channels
        nh, nw : input width and height
        kh, kw : kernel width and height
        ph, pw : height and width padding
        toc, tic : the tiling sizes of the output and input channels
    """
    X = te.placeholder(shape=(ic, nh, nw), name='X')
    K = te.placeholder(shape=(oc, ic, kh, kw), name='K')
    PaddedX = d2ltvm.padding(X, ph, pw) if ph * pw != 0 else X
    #Packed X
    assert ic % tic == 0 and oc % toc == 0
    PackedX = te.compute((ic // tic, nh, nw, tic), lambda ic_out, x, y, ic_in: PaddedX[ic_out * tic + ic_in, x, y], name='PackedX')
    PackedK = te.compute((oc // toc, ic // tic, kh, kw, tic, toc), 
                         lambda oc_out, ic_out, x, y, ic_in, oc_in: K[oc_out * toc + oc_in, ic_out * tic + ic_in, x, y], name='PackedK')
    
    return X, K, PaddedX, PackedX, PackedK

In [7]:
X, _, _, PackedX, _ = conv_pack(c, c, n, n, 1, 1, 0, 0, tc, tc)
mod = tvm.build(te.create_schedule(PackedX.op), [X, PackedX], target)
packed_x = tvm.nd.array(np.empty((c // tc, n, n, tc), dtype='float32'))
mod(tvm.nd.array(x), packed_x)
np.testing.assert_equal(packed_x.asnumpy(), y)